# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
#API Url
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f"{url}appid={weather_api_key}&units={units}&q=" 

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

602

In [4]:
city_data = pd.DataFrame(columns=['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed'])
for city  in cities:
    try:
        #city_url = weather_base_url  + "&q=" + city
        response = requests.get(query_url + city).json()
        city_data.loc[city,'City'] = response['name']
        city_data.loc[city,'Cloudiness'] = response['clouds']['all']
        city_data.loc[city,'Country'] = response['sys']['country']
        city_data.loc[city,'Date'] = response['sys']['sunset']
        city_data.loc[city,'Humidity'] = response['main']['humidity']
        city_data.loc[city,'Lat'] = response['coord']['lat']
        city_data.loc[city,'Lng'] = response['coord']['lon']
        city_data.loc[city,'Max Temp'] = response['main']['temp_max']
        city_data.loc[city,'Wind Speed'] = response['wind']['speed']
    except: KeyError
    pass
city_data = city_data.reset_index()
del city_data['index']
city_data.to_csv ('city_data.csv', index = None, header=True)
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ancud,40,CL,1580084616,59,-41.87,-73.82,18,2.6
1,Port Augusta,75,AU,1580118923,82,-32.5,137.77,20,4.6
2,Port Hardy,75,CA,1580087401,81,50.7,-127.42,9,3.6
3,Punta Arenas,20,CL,1580086239,47,-53.15,-70.92,14,12.9
4,Arraial do Cabo,0,BR,1580074649,69,-22.97,-42.03,27,5.7


In [ ]:
#Plotting Data Lat vs Temp
title = f"City Latitude vs Max Temperature"
plt.scatter(city_data["Lat"], city_data["Max Temp"], marker = "o", color="red")
plt.title(title, size=15)
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (C)")
plt.grid(True)
plt.savefig(f"{title}.png")

In [ ]:
#Plotting Data Lat vs Humidity
title = "City Latitude vs Humidity"
plt.scatter(city_data["Lat"], city_data["Humidity"], marker = "o", color="red")
plt.title(title, size=15)
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)
plt.savefig(f"{title}.png")

In [ ]:
#Plotting Data Lat vs Cloudiness
title = "City Latitude vs Cloudiness"
plt.scatter(city_data["Lat"], city_data["Cloudiness"], marker = "o", color="red")
plt.title(title, size=15)
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)
plt.savefig(f"{title}.png")

In [ ]:
#Plotting Data Lat vs Wind Speed
title = "City Latitude vs Wind Speed"
plt.scatter(city_data["Lat"], city_data["Wind Speed"], marker = "o", color="red")
plt.title(title, size=15)
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (kph)")
plt.grid(True)
plt.savefig(f"{title}.png")

In [ ]:
#North/South Regression
city_data_north = city_data.drop(city_data[city_data["Lat"] <= 0].index)
city_data_south = city_data.drop(city_data[city_data["Lat"] > 0].index)

In [ ]:
#North Hem. Stats
title = "North Hem.  Temp vs Latitude Linear Regression"
x = city_data_north["Max Temp"]
y = city_data_north["Lat"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#North Hem. Stats
title = "North Hem.  Humidity vs Lat Linear Regression"
x = city_data_north["Humidity"]
y = city_data_north["Lat"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#North Hem. Stats
title = "North Hem.  Cloudiness vs Lat Linear Regression"
x = city_data_north["Cloudiness"]
y = city_data_north["Lat"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#North Hem. Stats
title = "North Hem.  Wind Speed vs Lat Linear Regression"
x = city_data_north["Wind Speed"]
y = city_data_north["Lat"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#South Hem. Stats
title = "South Hem.  Temperature vs Lat Linear Regression"
x = city_data_south["Lat"]
y = city_data_south["Max Temp"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#South Hem. Stats
title = "South Hem.  Humidity vs Lat Linear Regression"
x = city_data_south["Lat"]
y = city_data_south["Humidity"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#South Hem. Stats
title = "South Hem.  Cloudiness vs Lat Linear Regression"
x = city_data_south["Lat"]
y = city_data_south["Cloudiness"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()

In [ ]:
#South Hem. Stats
title = "South Hem.  Wind Speed vs Lat Linear Regression"
x = city_data_south["Lat"]
y = city_data_south["Wind Speed"]
slope, intercept, r_value, p_value, std_err = linregress(x.astype(float), y.astype(float))
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(0,-40),fontsize=15,color="red")
plt.title(title)
plt.xlabel('Latitude')
plt.ylabel("Temperature (C)")
plt.savefig(f"{title}.png")
print(f"The r-squared is: {r_value}")
plt.show()